<a href="https://colab.research.google.com/github/gregorio-saporito/Spark-AMD/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding similar items: StackSample
Gregorio Luigi Saporito - DSE (2020-2021)

In [25]:
# optional remove files
# !rm kaggle.json
# !rm Questions.csv
# !rm Body.csv
# !rm -rf spark-3.1.1-bin-hadoop2.7

### Upload to session storage the Kaggle API token

In [26]:
from google.colab import files
uploaded = files.upload()

import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

Saving kaggle.json to kaggle.json


### Download the dataset through the Kaggle API

In [27]:
# access permissions with the API token
!chmod 600 /content/kaggle.json
!kaggle datasets download -d stackoverflow/stacksample
!unzip \*.zip && rm *.zip
# remove datasets which are not needed
!rm Answers.csv
!rm Tags.csv

100% 1.10G/1.11G [00:08<00:00, 129MB/s]
100% 1.11G/1.11G [00:08<00:00, 136MB/s]
Archive:  stacksample.zip
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


### Spark environment setup

In [28]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz 
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!rm /content/spark-3.1.1-bin-hadoop2.7.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

import findspark
findspark.init("spark-3.1.1-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

import pyspark
type(spark)

sc = spark.sparkContext

### Load Dataset
Spark reads files line by line for performance reasons and CSVs with newline characters cause problems for the parser. In this case the Body column of the file "Questions.csv" has characters like "\n" and "\r" which compromise the correct loading of the dataset. To solve the problem a third party parser capable of coping with this issue was used and a .csv file without newline characters is written on Disk. An alternative in a production scenario would be storing the files in a database. The RAM used for the parser is then freed up to save space. The new .csv file is then correctly loaded with Spark.

In [29]:
import pandas as pd
parsed = pd.read_csv("Questions.csv", encoding="ISO-8859-1", usecols=["Body"])
parsed['Body'] = parsed['Body'].str.replace(r'\n|\r', '')
parsed.to_csv("Body.csv", index=False)
del parsed

In [138]:
df = spark.read.load("Body.csv", format="csv",
                     inferSchema="true", header="true")
df

DataFrame[Body: string]

In [137]:
df.show(10)

+---------------------------------------+
|regexp_replace(lower(Body), <.*?>, , 1)|
+---------------------------------------+
|                   "i've written a d...|
|                   "are there any re...|
|                   has anyone got ex...|
|                   this is something...|
|                   "i have a little ...|
|                   i am working on a...|
|                   i've been writing...|
|                   "i wonder how you...|
|                   i would like the ...|
|                   i'm trying to mai...|
+---------------------------------------+
only showing top 10 rows



### Dataset Cleaning

In [32]:
# check for missing values
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+
|Body|
+----+
|   0|
+----+



In [139]:
from pyspark.sql.functions import col, lower, regexp_replace, split

def clean_html(x):
  x = lower(x)
  x = regexp_replace(x, '<.*?>', '')
  return x

df = df.select(clean_html(col("Body")).alias("Body"))

In [141]:
df.show(10, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------